In [ ]:
import pandas as pd
import altair as alt
from IPython.display import display, Markdown

# ==============================================================================
# 1. SETUP DE TEMA PERSONALIZADO (Basado en el CSS)
# ==============================================================================

# Colores y fuentes de tu CSS
primary_dark = "#01385F"
secondary_light = "#8fc7ff"
regular_font = 'Rubik-Regular, sans-serif'
bold_font = 'Rubik-ExtraBold, sans-serif'

def custom_webstory_theme():
    return {
        "config": {
            "view": {"stroke": primary_dark, "fill": "#FFFFFF", "strokeWidth": 3},
            "title": {"font": bold_font, "fontSize": 20, "color": primary_dark, "anchor": "middle"},
            "axis": {
                "titleFont": regular_font, "titleFontSize": 14, "titleColor": primary_dark,
                "labelFont": regular_font, "labelFontSize": 12, "labelColor": primary_dark,
                "gridColor": primary_dark + "30", "domainColor": primary_dark, "tickColor": primary_dark
            },
            "header": {
                "titleFont": bold_font, "titleFontSize": 14, "color": primary_dark,
                "labelFont": regular_font, "labelFontSize": 12, "labelColor": primary_dark
            },
            "legend": {
                "titleFont": bold_font, "titleFontSize": 14, "color": primary_dark,
                "labelFont": regular_font, "labelFontSize": 12, "labelColor": primary_dark, "padding": 10
            },
            "range": {
                "category": [secondary_light, primary_dark]
            }
        }
    }

alt.themes.register("webstory_style", custom_webstory_theme)
alt.themes.enable("webstory_style")


# ==============================================================================
# 2. CARGA Y PRE-PROCESAMIENTO DE DATOS
# ==============================================================================

# Cargar el archivo de datos (Asegúrate de que el archivo esté subido a Colab)
try:
    df = pd.read_csv("Florencia_database_utilizada 2.csv", delimiter=';', encoding='latin1')
except FileNotFoundError:
    display(Markdown("---ERROR: Archivo CSV no encontrado. Por favor, sube 'Florencia_database_utilizada 2.csv' a Google Colab.---"))
    exit()

events_to_analyze = ['Salto Largo', 'Salto Garrocha']
df_filtered = df[df['Prueba'].isin(events_to_analyze)].copy()

# Datos de participación por año (para gráficos de líneas y puntos)
participation_df = df_filtered.groupby(['Prueba', 'A\x96o', 'Categoria'])['Nombre Atleta'].nunique().reset_index()
participation_df.rename(columns={'Nombre Atleta': 'Participacion', 'A\x96o': 'Año'}, inplace=True)

# Datos de participación total (para cálculos de cierre y gráfico atómico)
vis_data = df_filtered.groupby(['Prueba', 'Categoria'])['Nombre Atleta'].nunique().reset_index()
vis_data.rename(columns={'Nombre Atleta': 'Total_Participacion'}, inplace=True)
participation_pivot = vis_data.pivot(index='Categoria', columns='Prueba', values='Total_Participacion').fillna(0)


# ==============================================================================
# 3. CÁLCULOS PARA PREGUNTAS DE CIERRE
# ==============================================================================

# Q1: Magnitud de la Brecha (Escolar)
escolar_data = participation_pivot[participation_pivot.index.str.contains('Escolar')]
total_largo_escolar = escolar_data['Salto Largo'].sum()
total_garrocha_escolar = escolar_data['Salto Garrocha'].sum()
brecha_escolar = total_largo_escolar - total_garrocha_escolar

# Q2: Distribución de la Escasez (Garrocha)
garrocha_data = participation_pivot[['Salto Garrocha']]
min_garrocha_categoria = garrocha_data['Salto Garrocha'].idxmin()

# Q3: Comparación de Género (Proporcionalidad)
masc_largo = participation_pivot[participation_pivot.index.str.contains('Masculino')]['Salto Largo'].sum()
masc_garrocha = participation_pivot[participation_pivot.index.str.contains('Masculino')]['Salto Garrocha'].sum()
ratio_masc = masc_largo / masc_garrocha if masc_garrocha > 0 else float('inf')

fem_largo = participation_pivot[participation_pivot.index.str.contains('Femenino')]['Salto Largo'].sum()
fem_garrocha = participation_pivot[participation_pivot.index.str.contains('Femenino')]['Salto Garrocha'].sum()
ratio_fem = fem_largo / fem_garrocha if fem_garrocha > 0 else float('inf')


# ==============================================================================
# 4. FUNCIONES DE GENERACIÓN DE GRÁFICOS
# ==============================================================================

def generate_line_chart(df_data, gender_filter, title_suffix, color_scale_domain):
    """Genera el gráfico de líneas y puntos con el tema personalizado."""

    df_filtered_gender = df_data[df_data['Categoria'].str.contains(gender_filter)].copy()

    color_scale = alt.Scale(
        domain=color_scale_domain,
        range=[primary_dark, secondary_light] # Adulto (Oscuro) y Escolar (Claro)
    )

    base = alt.Chart(df_filtered_gender).encode(
        x=alt.X('Año:O', axis=alt.Axis(title='Año')),
        y=alt.Y('Participacion:Q', axis=alt.Axis(title='Número de Atletas Únicos')),
        color=alt.Color('Categoria:N', title='Categoría', scale=color_scale)
    )

    line = base.mark_line().encode(
        tooltip=['Año:O', 'Categoria:N', alt.Tooltip('Participacion:Q', title='Participación')]
    )

    points = base.mark_point(filled=True, size=80, strokeWidth=1, opacity=0.9).encode(
        tooltip=['Año:O', 'Categoria:N', alt.Tooltip('Participacion:Q', title='Participación')]
    )

    chart = (line + points).facet(
        column=alt.Column('Prueba:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Prueba')
    ).properties(
        title=f'Participación de Atletas {title_suffix}: Salto Largo vs. Garrocha por Año'
    ).interactive()

    return chart

def generate_atomic_chart(df_data):
    """Genera el gráfico de barras agrupadas para el cierre."""

    color_scale = alt.Scale(
        domain=['Salto Largo', 'Salto Garrocha'],
        range=[secondary_light, primary_dark] # Largo (Claro) vs Garrocha (Oscuro)
    )

    base = alt.Chart(df_data).encode(
        y=alt.Y('Total_Participacion:Q', title='Total de Atletas Únicos (2023-2025)'),
        x=alt.X('Prueba:N', title='Prueba', axis=None),
        color=alt.Color('Prueba:N', scale=color_scale, title='Prueba'),
        tooltip=['Categoria:N', 'Prueba:N', alt.Tooltip('Total_Participacion:Q', title='Participación Total')]
    )

    bars = base.mark_bar().properties(height=300)

    text = base.mark_text(
        align='center',
        baseline='bottom',
        dy=-5,
        color=primary_dark
    ).encode(
        text=alt.Text('Total_Participacion:Q', format=',.0f'),
        order=alt.Order('Total_Participacion', sort='descending')
    )

    chart = (bars + text).properties(
        title='Diferencia de Participación Absoluta por Categoría (Total Atletas 2023-2025)'
    ).facet(
        column=alt.Column('Categoria:N', header=alt.Header(titleOrient="top", labelOrient="bottom", title='Categoría'))
    ).resolve_scale(
        x='independent'
    ).interactive()

    return chart

# ==============================================================================
# 5. CRÓNICA Y PRESENTACIÓN DE RESULTADOS (Texto Ajustado)
# ==============================================================================

# Texto Introductorio
cronica_markdown = f"""
#Una mejora de resultados que aún no se refleja en el salto con garrocha

El reporte de resultados, tanto en lanzamientos, como en 800 metros planos, no refleja la infraestructura y la participación en las pruebas de salto. Es por esto que analizamos la magnitud y distribución de la escasez, utilizando los datos de participación absoluta.

---

## **Las cifras de la desigualdad**

El siguiente gráfico compara la participación total en **salto largo** y **salto con garrocha** por categoría, revela las cifras exactas que sustentan la necesidad urgente de infraestructura, tal como las pistas de alta competencia públicas, que hoy se limitan a regiones como Limache en la Región de Valparaíso (Fuente: Limache inauguró moderna Pista Atlética de Alta Competencia.pdf) y las de la Región Metropolitana, como el Estadio Nacional reconstruido para los Juegos Panamericanos Santiago 2023.

---
### Participación de atletas por año

A continuación, la evolución de la participación anual por género:
"""

# Texto de Cierre y Respuestas a Preguntas
cierre_markdown = f"""
---
## **Análisis de la brecha: preguntas y respuestas**

A continuación, las respuestas a nuestras preguntas de cierre, basadas en el análisis de la participación total, entre 2023 y 2025, junto a su visualización:

1.  **Magnitud de la brecha en categoría escolar**
    * **Pregunta:** ¿Cuál es la diferencia absoluta de atletas únicos entre salto largo y salto garrocha en las categorías Escolares masculina y femenina?
    * **Respuesta:** En la categoría Escolar, hay **{int(brecha_escolar)} atletas únicos más** participando en salto largo que en salto con garrocha.

2.  **Distribución de la escasez**
    * **Pregunta:** ¿En qué categoría se concentra la menor participación absoluta en salto con garrocha a nivel nacional?
    * **Respuesta:** La menor participación absoluta en salto con garrocha se concentra en la categoría **{min_garrocha_categoria}**, con solo **{int(garrocha_data.loc[min_garrocha_categoria]['Salto Garrocha'])} atletas únicos** en el periodo analizado.

3.  **Comparación de Género**
    * **Pregunta:** ¿La brecha de participación entre salto largo y salto garrocha es proporcionalmente mayor en categorías masculinas o femeninas?
    * **Respuesta:** La diferencia de participación es **proporcionalmente mayor** en las categorías **Femeninas**, rango de **{ratio_fem:.2f} veces más** participantes en largo que en garrocha, en comparación con las categorías Masculinas, que es **{ratio_masc:.2f} veces** más.

> Las categorías femeninas, especialmente la adulta, enfrentan la mayor escasez relativa de atletas en salto con garrocha. Esto refuerza el argumento de que la falta de acceso a instalaciones especializadas de nivel superior, como la nueva pista del Estadio Nacional, anunciada por Fedachi en 2023, que sí soporta estas pruebas, impacta de manera más aguda en el desarrollo de esta especialidad en el país.

"""

# ==============================================================================
# 6. ORDEN DE DESPLIEGUE EN COLAB
# ==============================================================================

# 6.1 Mostrar la Crónica Introductoria
display(Markdown(cronica_markdown))

# 6.2 Generar y mostrar el Gráfico Masculino (Líneas y Puntos)
display(generate_line_chart(
    participation_df,
    'Masculino',
    'Masculinos',
    ['Escolar Masculino', 'Adulto Masculino']
))

# 6.3 Generar y mostrar el Gráfico Femenino (Líneas y Puntos)
display(generate_line_chart(
    participation_df,
    'Femenino',
    'Femeninas',
    ['Escolar Femenino', 'Adulto Femenino']
))

# 6.4 Mostrar el Cierre y las Preguntas de Investigación
display(Markdown(cierre_markdown))

# 6.5 Generar y mostrar el Gráfico Atómico de Cierre
display(generate_atomic_chart(vis_data))



#Una mejora de resultados que aún no se refleja en el salto con garrocha

El reporte de resultados, tanto en lanzamientos, como en 800 metros planos, no refleja la infraestructura y la participación en las pruebas de salto. Es por esto que analizamos la magnitud y distribución de la escasez, utilizando los datos de participación absoluta.

---

## **Las cifras de la desigualdad**

El siguiente gráfico compara la participación total en **salto largo** y **salto con garrocha** por categoría, revela las cifras exactas que sustentan la necesidad urgente de infraestructura, tal como las pistas de alta competencia públicas, que hoy se limitan a regiones como Limache en la Región de Valparaíso (Fuente: Limache inauguró moderna Pista Atlética de Alta Competencia.pdf) y las de la Región Metropolitana, como el Estadio Nacional reconstruido para los Juegos Panamericanos Santiago 2023.

---
### Participación de atletas por año

A continuación, la evolución de la participación anual por género:


alt.FacetChart(...)

alt.FacetChart(...)


---
## **Análisis de la brecha: preguntas y respuestas**

A continuación, las respuestas a nuestras preguntas de cierre, basadas en el análisis de la participación total, entre 2023 y 2025, junto a su visualización:

1.  **Magnitud de la brecha en categoría escolar**
    * **Pregunta:** ¿Cuál es la diferencia absoluta de atletas únicos entre salto largo y salto garrocha en las categorías Escolares masculina y femenina?
    * **Respuesta:** En la categoría Escolar, hay **25 atletas únicos más** participando en salto largo que en salto con garrocha.

2.  **Distribución de la escasez**
    * **Pregunta:** ¿En qué categoría se concentra la menor participación absoluta en salto con garrocha a nivel nacional?
    * **Respuesta:** La menor participación absoluta en salto con garrocha se concentra en la categoría **Adulto Femenino**, con solo **9 atletas únicos** en el periodo analizado.

3.  **Comparación de Género**
    * **Pregunta:** ¿La brecha de participación entre salto largo y salto garrocha es proporcionalmente mayor en categorías masculinas o femeninas?
    * **Respuesta:** La diferencia de participación es **proporcionalmente mayor** en las categorías **Femeninas**, rango de **1.86 veces más** participantes en largo que en garrocha, en comparación con las categorías Masculinas, que es **1.70 veces** más.

> Las categorías femeninas, especialmente la adulta, enfrentan la mayor escasez relativa de atletas en salto con garrocha. Esto refuerza el argumento de que la falta de acceso a instalaciones especializadas de nivel superior, como la nueva pista del Estadio Nacional, anunciada por Fedachi en 2023, que sí soporta estas pruebas, impacta de manera más aguda en el desarrollo de esta especialidad en el país.



alt.FacetChart(...)